In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
tqdm.pandas()

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression, SGDClassifier

Здесь выполняется *подготовка* и *загрузка* **стоп-слов**:

In [ ]:
import nltk
import spacy

stopwords_nltk = nltk.corpus.stopwords.words('russian') # лист русских стоп-слов
stopwords_nltk_en = nltk.corpus.stopwords.words('english')
stopwords_nltk.extend(stopwords_nltk_en) # если есть текста на английском

new_stop = ['здравствовать', 'подсказать', 'сказать', "пожалуйста", "спасибо",  "благодарить", "извинить",
            'вопрос','тема', "ответ", "ответить", "почему", "что",
            'которая', 'которой', 'которую', 'которые', 'который', 'которых', 'это', "мочь",
            'вообще', "всё", "весь", "ещё", "просто", 'точно', "хотя", "именно", 'неужели',]
stopwords_nltk.extend(new_stop)

lemmatizer = spacy.load('ru_core_news_md', disable = ['parser', 'ner'])

In [ ]:
# сохраним стоп слова, чтобы не загружать ради них nltk
with open("model/stopwords.txt", "w") as f:
    for name in stopwords_nltk[:-1]:
        f.write(f"{name}\n")
        f.write(stopwords_nltk[-1])


Здесь находятся *функции*, служащие для *подготовки текста* для его дальнейшей передачи в **языковую модель**:

In [ ]:
# очистка текста
def full_clean(text):
    '''подготовка текста к подаче в модель для каждого текста (через applay)'''
    text=re.sub(r"[^a-zA-Zа-яА-ЯёЁ0-9#]", " ", text)
    text = text.lower()
    text = re.sub(" +", " ", text).strip() # оставляем только 1 пробел
    # токены для моделей
    tokens = [token.lemma_ for token in lemmatizer(text) if token.lemma_ not in stopwords_nltk]
    # для tfidf на вход текст
    text = " ".join(tokens)
    return text, tokens


def preprocess_text(df):
    '''подготовка текста к подаче в модель колонкой'''
    new_corpus=[]
    new_tokens=[]

    for text in tqdm(df):
        text, tokens = full_clean(text)
        new_corpus.append(text)
        new_tokens.append(tokens)

    return new_corpus, new_tokens

In [ ]:
def tfidf_fit(train=None, test=None, tfidf=True, ngram_range=(1, 1), max_features=1000, save=False):
    # на вход текст
    # min_df : игнорируются термины, частота употребления которых строго ниже заданного порога.
    # max_df : игнорируются термины, частота которых строго превышает заданный порог
    if test:
        data = pd.concat([train, test])
    else:
        data = train
    if tfidf:
        model = TfidfVectorizer(ngram_range=(1, 1), max_features=max_features, analyzer='word', #max_df = 0.9,
                            lowercase = False, sublinear_tf=True)
    else:
        model = CountVectorizer(max_features=max_features)
    # тренировка
    model.fit(data)

    # сохранение натренированной модели для приложения
    if save:
        joblib.dump(model, 'tfidf.pkl')

    return model

def tfidf_embeding(model=None, df=None):
    '''Преобразование текста в мешок слов'''
    if model==None:
        # загрузить если нет
        model = joblib.load('tfidf.pkl')
    else:
        model=model
    X = model.transform(df)

    return X.toarray()

Здесь мы загружаем *тренировочный датафрейм*, дополняем его сторонней аргументацией, форматируем, и сохраняем его в файл:

In [ ]:
train = pd.read_csv('train_SecondPilot/train_data.csv', sep=',', index_col=None)
train.head()

,Question,Category,answer_class
0,"Что делать, если я хочу изменить группу или сп...",Перевод/ запись в группу,11
1,Из чего состоит основное обучение в личном каб...,Портал,15
2,Что входит в программу помощи с трудоустройств...,Трудоустройство,27
3,"Подскажите расписание каникул в 2024, пожалуйста",Расписание,21
4,Какие инструменты и ресурсы вы предоставляете ...,Трудоустройство,25


In [ ]:
# дополнительные аргументации в разных файлах
frame_list = ['data/train_data_new0.csv', 'data/train_data_new1-.csv', 'data/train_data_spell.csv',
              'data/call_operator.csv', 'data/19_osn.csv', "data/train_ayg.csv", "data/train_data_new_new.csv"]
train = pd.concat(map(pd.read_csv, frame_list), axis=0, ignore_index=True)
train.drop_duplicates(subset=['Question'], inplace=True, ignore_index=True)
train = train[['Question', 'Category', 'answer_class']]
train.dropna(how='any', axis=0, ignore_index=True, inplace=True)
train['answer_class'] = train['answer_class'].astype(int)
train.head()

,Question,Category,answer_class
0,Где можно узнать о документах или сертификатах...,Документы,0
1,Какой документ я получу после обучения?,Документы,0
2,Какие аккредитации или удостоверения я могу ож...,Документы,0
3,Как происходит выдача дипломов студентам?,Документы,0
4,"Где можно узнать о том, как получить документы...",Документы,0


In [ ]:
train.shape

(1369, 3)

In [ ]:
train[['Question', 'Category', 'answer_class']].to_csv('data/train_1.csv', sep=',', index=False)

Добавляем *отдельный класс_ответа*, при котором бот будет переключать нас **на оператора**:

In [ ]:
train_answer_class = pd.read_csv('train_SecondPilot/answer_class.csv', sep=',', index_col=None)
# добавление нового класса для переключения на оператора
train_answer_class.loc[len(train_answer_class.index )] = [30, 'Переключаю на оператора']
train_answer_class.head()

,answer_class,Answer
0,0,После успешного прохождения выпускных испытани...
1,1,"Чтобы получить итоговый документ, нужно сдать ..."
2,2,"Можем его выдать, если вы:\n\nоплатили обучени..."
3,3,"Можем его выдать, если вы:\n\nоплатили обучени..."
4,4,Диплом или удостоверение отправим бесплатно По...


In [ ]:
train_answer_class.shape

(31, 2)

Далее, происходит **обработка текста**, а именно: создание новых столбцов с токенами и "чистым текстом":

In [ ]:
train['text_clean'], train['tokens']=preprocess_text(train['Question'])

  0%|          | 0/1369 [00:00<?, ?it/s]

In [ ]:
train.head()

,Question,Category,answer_class,text_clean,tokens
0,Где можно узнать о документах или сертификатах...,Документы,0,узнать документ сертификат завершение обучение,"[узнать, документ, сертификат, завершение, обу..."
1,Какой документ я получу после обучения?,Документы,0,документ получить обучение,"[документ, получить, обучение]"
2,Какие аккредитации или удостоверения я могу ож...,Документы,0,аккредитация удостоверение ожидать получить за...,"[аккредитация, удостоверение, ожидать, получит..."
3,Как происходит выдача дипломов студентам?,Документы,0,происходить выдача диплом студент,"[происходить, выдача, диплом, студент]"
4,"Где можно узнать о том, как получить документы...",Документы,0,узнать получить документ выпуск,"[узнать, получить, документ, выпуск]"


Здесь происходит *обучение TF-IDF* модели на основе ранее созданных колонок, а также дальнейшее **сохранение признаков (слов)** из *обученной модели*:

In [ ]:
tfidf = tfidf_fit(train=train['text_clean'], tfidf=True, max_features=1100)
feature_names = tfidf.get_feature_names_out()
len(feature_names)

1027

In [ ]:
# сохранить модель
joblib.dump(tfidf, 'model/tfidf.pkl')
# загрузить если нет

['model/tfidf.pkl']

In [ ]:
tfidf_embed = tfidf_embeding(model=tfidf, df=train['text_clean'])

Сохраняем *натренированную модель* в отдельный **файл**:

In [ ]:
# сохраним классы и эмбединги вопросов в один файл
classes = np.array(train.answer_class)
temp = {'classes': classes, 'tfidf_embed': tfidf_embed}
joblib.dump(temp, 'model/train.pkl')

['model/train.pkl']

In [ ]:
# обновим классы в ответах
train_answer_class.to_csv('model/answer_class.csv', sep=',', index=True)

функция **find_similarity()** ищет наиболее схожий с переданным в неё текст:

In [ ]:
def find_similarity(query, embeddings, train, top_k=3):
    # самые близкие вопросы из трейна
    # возвращает класс, схожесть
    query_embedding = tfidf_embeding(model=tfidf, df=[full_clean(query)[0]])[0]
    cos_similarities = cosine_similarity(query_embedding.reshape(1, -1), np.array(embeddings))
    sorted_indices = np.argsort(cos_similarities[0])[::-1]
    classes = np.array(train.answer_class)
    top_class = [classes[idx] for i, idx in enumerate(sorted_indices[0:top_k])]

    return top_class[0], cos_similarities[0][sorted_indices[0]]

In [ ]:
query = "Я хочу изменить группу, какой мой порядок действий?"
answer_class, similarity = find_similarity(query, tfidf_embed, train, top_k=3)

In [ ]:
answer = train_answer_class[train_answer_class.answer_class==answer_class]['Answer'].values[0]
answer

'Мы можем перевести вас в другую группу в рамках срока обучения и дополнительных 6 месяцев сверху. Срок обучения отсчитывается с даты оплаты обучения.\nКоличество переводов зависит от срока программы:\n\nЕсли ваш продукт предусматривает возможность выбора специализации:\n12 месяцев — 1 перевод на каждый блок обучения, суммарно 3 перевода на весь срок обучения.\n24 месяца — 1 перевод на каждый блок обучения, суммарно 4 перевода на весь срок обучения.\n36 месяцев — 1 перевод на каждый блок обучения, суммарно 4 перевода на весь срок обучения.\nЕсли ваш продукт не предусматривает возможность выбора специализации:\n6 месяцев — 1 перевод на все время обучения.\n9 месяцев — 1 перевод на все время обучения.\n12 месяцев — 2 перевода на все время обучения.\nУзнать о сроках обучения и специализациях можно из программы обучения на странице вашего продукта.'

In [ ]:
answer_class

11

In [ ]:
train_answer_class[train_answer_class.answer_class==11]['Answer'].values[0]

'Мы можем перевести вас в другую группу в рамках срока обучения и дополнительных 6 месяцев сверху. Срок обучения отсчитывается с даты оплаты обучения.\nКоличество переводов зависит от срока программы:\n\nЕсли ваш продукт предусматривает возможность выбора специализации:\n12 месяцев — 1 перевод на каждый блок обучения, суммарно 3 перевода на весь срок обучения.\n24 месяца — 1 перевод на каждый блок обучения, суммарно 4 перевода на весь срок обучения.\n36 месяцев — 1 перевод на каждый блок обучения, суммарно 4 перевода на весь срок обучения.\nЕсли ваш продукт не предусматривает возможность выбора специализации:\n6 месяцев — 1 перевод на все время обучения.\n9 месяцев — 1 перевод на все время обучения.\n12 месяцев — 2 перевода на все время обучения.\nУзнать о сроках обучения и специализациях можно из программы обучения на странице вашего продукта.'

In [ ]:
query = "Я хочу изменить группу, какой мой порядок действий?"

In [ ]:
def predict(query):
    answer_class, similarity = find_similarity(query, tfidf_embed, train, top_k=3)
    answer = train_answer_class[train_answer_class.answer_class==answer_class]['Answer'].values[0]

    return answer_class, answer, round(similarity, 2)

In [ ]:
answer_class, answer, similarity = predict(query)

In [ ]:
similarity

0.55